# Extração de sabores

#### Setup

In [2]:
# Create the connection with Google Cloud
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

def create_connection():
        # Set the credentials and create the connection
        key_path = "../usp-mba-dsa-tcc-4277103d9155.json"

        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes=["https://www.googleapis.com/auth/bigquery"]
        )

        client = bigquery.Client(
            credentials=credentials,
            project=credentials.project_id,
        )
        return client

client = create_connection()

In [3]:
# Perform a query.
QUERY = ("""
    SELECT DISTINCT offer_id, LOWER(Descricao) Descricao
    FROM `usp-mba-dsa-tcc.ecommerce_offers.vw_dim_offers`
""")
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

offers = rows.to_dataframe()

In [4]:
offers.head(2)

,offer_id,Descricao
0,3ce9bbd3b162e2d44436c0ae34932e6dd5bf02e8143a88...,ração three dogs adultos minis e pequenos fran...
1,d3363c5e0c80eb6950031df1736ba4a05b7680025d2687...,ração hill's science diet canino cães adulto 1...


#### Lista bruta de tokens

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('portuguese'))

In [13]:
palavras_oferta = offers['Descricao'].apply(word_tokenize) 

In [16]:
words = [elemento for sublista in palavras_oferta for elemento in sublista]

In [19]:
import re

def remove_stopwords_and_numbers(words):
    filtered_tokens = []
    for token in words:
        # Verifique se o token não é uma stop word e não é um número
        if token.lower() not in stop_words and not re.match(r'^\d+(?:\.\d+)?$', token):
            filtered_tokens.append(token)
    return filtered_tokens

tokens_sem_stopwords_e_numeros = remove_stopwords_and_numbers(words)
print(len(tokens_sem_stopwords_e_numeros))
tokens_sem_stopwords_e_numeros

48303


['ração',
 'three',
 'dogs',
 'adultos',
 'minis',
 'pequenos',
 'frango',
 'cenoura',
 'kg',
 'ração',
 'hill',
 "'s",
 'science',
 'diet',
 'canino',
 'cães',
 'adulto',
 '11+',
 'pequenos',
 'mini',
 '2,4',
 'kg',
 'ração',
 'seca',
 'three',
 'cats',
 'super',
 'premium',
 'frango',
 'gatos',
 'filhotes',
 '-',
 '3kg',
 'ração',
 'fórmula',
 'natural',
 'pró',
 'cães',
 'adultos',
 'porte',
 'mini',
 'pequeno',
 'sabor',
 'frango',
 'arroz',
 'integral',
 'ração',
 'premiatta',
 'light',
 'cães',
 'adultos',
 'porte',
 'médio',
 'grande',
 'sabor',
 'frango',
 'arroz',
 'kg',
 'ração',
 'úmida',
 'guabi',
 'natural',
 'frango',
 ',',
 'cereias',
 'vegetais',
 'gatos',
 'castrados',
 '-',
 'g',
 'ração',
 'premier',
 'pet',
 'raças',
 'específicas',
 'yorkshire',
 'adulto',
 '-',
 'kg',
 'ração',
 'monello',
 'select',
 'cães',
 'sênior',
 'sabor',
 'frango',
 'arroz',
 'ração',
 'naturalis',
 'lifebites',
 'cães',
 'adultos',
 'porte',
 'pequeno',
 'sabor',
 'peru',
 ',',
 'frango'

In [ ]:
# Salvar lista de tokens
df = pd.DataFrame(tokens_sem_stopwords_e_numeros)
df.to_csv('sabores.csv', index=False)

Após isso as palavras foram triadas manualmente, mantendo apenas aquelas realmente relacionadas com sabores dos produtos

#### Extração de sabores por oferta

In [20]:
# Carrega a lista de sabores
lista_sabores = list(set(pd.read_csv('sabores.csv')['words']))
len(lista_sabores)

109

In [21]:
from nltk.tokenize import word_tokenize
offers["Sabores"] = offers['Descricao'].apply(word_tokenize) 

In [22]:
# Filtrar sabores da lista
def filtrar_sabores(lista):
    lista_filtrada = [sabor.capitalize() for sabor in lista if sabor in lista_sabores]
    return sorted(lista_filtrada)

offers['Sabores'] = offers['Sabores'].apply(filtrar_sabores)

In [23]:
# Transformar listas de sabores em string
def transformar_lista_para_string(lista):
    # Verifica se a lista está vazia
    if not lista:
        return None
    
    # Se houver apenas um elemento, retorna esse elemento como uma string
    if len(lista) == 1:
        return lista[0]
    
    # Se houver mais de um elemento, formata a lista como uma string com "e" entre os dois últimos elementos
    return ', '.join(lista[:-1]) + ' e ' + lista[-1]

offers['Sabores'] = offers['Sabores'].apply(transformar_lista_para_string)

In [24]:
# Drop column Descricao
offers.drop('Descricao', axis=1, inplace=True)
offers.head(2)

,offer_id,Sabores
0,3ce9bbd3b162e2d44436c0ae34932e6dd5bf02e8143a88...,Cenoura e Frango
1,d3363c5e0c80eb6950031df1736ba4a05b7680025d2687...,None


#### Upload para o BQ

In [ ]:
# Salva dataframe como tabela no BQ (substituindo)
project_id = client.project

offers.to_gbq(
    project_id=project_id,
    destination_table='ecommerce_offers.tb_sabores',
    if_exists='replace'
)